In [1]:
from qpms import Particle, CTMatrix, lorentz_drude, EpsMuGenerator, TMatrixGenerator, BaseSpec, FinitePointGroup, ScatteringSystem, TMatrixInterpolator, EpsMu, dbgmsg_enable, dbgmsg_disable, dbgmsg_active, BesselType,eV, hbar, c
from qpms.symmetries import point_group_info
from qpms.cyewaldtest import scatsyswk_sswf_basis, scatsysw_eval_sswf
from qpms.cycommon import get_mn_y_scalar, get_nelem_scalar
import math
import logging
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)
import numpy as np
eh = eV/hbar
np.random.seed(666)
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Rectangle, Circle
from mpl_toolkits.axes_grid1 import AxesGrid

def cleanarray(a, atol=1e-10, copy=True):
    a = np.array(a, copy=copy)
    sieve = abs(a.real) < atol
    a[sieve] = 1j * a[sieve].imag
    sieve = abs(a.imag) < atol
    a[sieve] = a[sieve].real
    return a

/usr/lib/python3/dist-packages/sympy/printing/str.py:362: SymPyDeprecationWarning: 

Permutation.print_cyclic = True has been deprecated since SymPy 1.6.
Use init_printing(perm_cyclic=True) instead. See
https://github.com/sympy/sympy/issues/15201 for more info.

  SymPyDeprecationWarning(


In [2]:
dbgmsg_enable(2)
#part_radius = 80e-9

<DebugFlags.THREADS|MISC: 3>

In [28]:
#modefile = '/u/46/necadam1/unix/repo/qpms/misc/modes_basis300nm_0nm__0nm_400nm_n1_b-2_k(0_0)um-1_cn150.npz' # PARAM!
modefile='/l/necadam1/test_mode_fieldplots/cyl_r50nm_h50nm_p560nmx560nm_mAu_n1.52_b-2_k(0_0)um-1_L3_cn200.npz'
#modefile = '/l/necadam1/sqlat-Tommi/sph_r80nm_p580nm_m5.0_n1.52_b-3_k(0_0)um-1_L3_cn3100.npz'
data = np.load(modefile, allow_pickle=True)
meta = data['meta'][()]
list(data.keys())

['meta',
 'empty_freqs',
 'ss_positions',
 'ss_fullvec_poffsets',
 'ss_fullvec_psizes',
 'ss_bspecs_flat',
 'ss_lattice_basis',
 'ss_reciprocal_basis',
 'eigval',
 'eigval_inside_metric',
 'eigvec',
 'residuals',
 'eigval_err',
 'ranktest_SV',
 'iri',
 'blochvector',
 'inside_contour',
 'refractive_index_internal']

In [29]:
# initialise the medium and dummy t-matrix generators, so we can "reconstruct" the ScatSystem instance (only background medium is relevant here)
medium = EpsMuGenerator(meta['background'])
emg_dummy = EpsMuGenerator(10)
tmg_dummy = TMatrixGenerator.sphere(medium, emg_dummy, 50e-9)
try: default_omega = data['eigval'][0]
except IndexError as ie:
    raise ValueError("Data file contains no eigenvalues") from ie
    

In [5]:
# create ss, ssw; init wavevector to use with sswk constructor
particles = []
offsets = data['ss_fullvec_poffsets']
nelems = data['ss_fullvec_psizes']
for pi, pos in enumerate(data['ss_positions']):
    bspec = BaseSpec(data['ss_bspecs_flat'][offsets[pi]:offsets[pi]+nelems[pi]])
    particles.append(Particle(pos, tmg_dummy, bspec))
ss, ssw = ScatteringSystem.create(particles, medium, default_omega, latticebasis = data['ss_lattice_basis'])
wavevector = np.array([0., 0., 0.]) # it needs to be 3D for sswk "constructor", 
for i in range(len(meta['k'])):
    wavevector[i] = meta['k'][i]
    
# determine the "unit rectangle" size for plotting
lb = ss.lattice_basis
xmax = max(abs(lb[0,0]), abs(lb[1,0]))
ymax = max(abs(lb[0,1]), abs(lb[1,1]))
if (xmax > ymax):
    unitrect = (xmax, ss.unitcell_volume/xmax)
else:
    unitrect = (ss.unitcell_volume/ymax, ymax)

In [6]:
eigval_i = 0 # PARAM!
#omega = data['eigval'][eigval_i]
omega = default_omega.real + 0.01j * default_omega.real
fvc = data['eigvec'][eigval_i]
len_fvc = len(fvc)

In [7]:
ssw = ss(omega)
sswk = ssw._sswk(wavevector)

In [8]:
ucfactor = 2.2 # "unit cell coverage"; unitrect sizes will be multiplied by this# PARAM!
zfactor = 5 # z range size / sqrt(unitcell_volume) size aspect ratio
resolution = 60 # PARAM!
sqrt_ucvol = math.sqrt(ss.unitcell_volume)
gridsz = [math.ceil(resolution * unitrect[i] / sqrt_ucvol) for i in (0,1)] #x, y
gridsz.append(math.ceil(zfactor / ucfactor * resolution)) # z
gridextent = [[ -(ucfactor * unitrect[i])/2, (ucfactor * unitrect[i] / 2) ] for i in (0,1)]
gridextent.append( [-(zfactor * sqrt_ucvol) / 2, (zfactor * sqrt_ucvol) / 2] )
gridlines = [np.linspace(*gridextent[i], gridsz[i]) for i in range(3)]
gridspacings = [gridlines[i][1]-gridlines[i][0] for i in range(3)] # should be very similar

gridoffsets = [0.,0.,0.] # PARAM!

xygrid = np.stack(np.meshgrid(gridlines[0], gridlines[1], gridoffsets[2], indexing='ij')).T
xygrid.shape=xygrid.shape[1:]
xygrid_s = xygrid - np.array([*gridspacings[:2], 0])/2 # correction of pcolormesh coordinates
xzgrid = np.stack(np.meshgrid(gridlines[0], gridoffsets[1], gridlines[2], indexing='ij')).T
xzgrid.shape = (gridsz[2], gridsz[0], 3)
xzgrid_s = xzgrid - np.array([gridspacings[0], 0, gridspacings[2]])/2

#for imshow
lenscale = 1000e-9 #PARAM!
xyextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(1,0),(1,1)]]
xzextent = [gridextent[i][j]/lenscale for (i,j) in [(0,0),(0,1),(2,0),(2,1)]]

In [9]:
logging.info("Start calculating xy field basis")
xy_basis = sswk.scattered_field_basis(xygrid)
#logging.info("Start calculating xz field basis")
#xz_basis = sswk.scattered_field_basis(xzgrid)
xz_basis = np.zeros(((137, 60, 30, 3)), dtype=complex)
logging.info("Start calculating xy sswf basis")
xy_sswf = scatsyswk_sswf_basis(sswk, xygrid)
logging.info("Basis fields done")

2020-11-23 15:36:28,486 Start calculating xy field basis
2020-11-23 15:36:33,388 Start calculating xy sswf basis
2020-11-23 15:36:55,995 Basis fields done


In [16]:
xy_sswf.shape, xy_basis.shape, max(bspec.l())

((60, 60, 1, 64), (60, 60, 30, 3), 3)

In [21]:
cutoff_ratio_default = 0.03
def cutfield(a, cutoff_ratio=cutoff_ratio_default):
    vmin = np.nanquantile(a, cutoff_ratio)
    vmax = np.nanquantile(a, 1-cutoff_ratio)
    return np.clip(a, vmin, vmax)

In [25]:
plotfile = "/tmp/pokus_sswf_basis_ewald.pdf" # PARAM
lMax_sswf = max(bspec.l()*2+1)
nelem = get_nelem_scalar(lMax_sswf)
m, l = get_mn_y_scalar(lMax_sswf)
with PdfPages(plotfile) as pdf:
  for y in range(nelem):
    xyf = cleanarray(xy_sswf[...,0,y])
    
    f = plt.figure(figsize=(7,4))
    f.suptitle("(%d, %d)" % (l[y], m[y]))
    grid = AxesGrid(f, 111,  # similar to subplot(111)
                nrows_ncols=(1, 2),
                axes_pad=(0.45, 0.65),
                label_mode="L",
                share_all=False,
                cbar_location="right",
                cbar_mode="each",
                cbar_size="7%",
                cbar_pad="2%",
                )
    
    # xy part
    ax = grid[0:2]
    cbax = grid.cbar_axes[0:2]

    for axi in range(2):
        ax[axi].set_xlim(xyextent[:2])
        ax[axi].set_ylim(xyextent[2:])
    flim = max(np.amax(cutfield(abs(xyf[...].real))), np.amax(cutfield(abs(xyf[...].imag)))) #this is stupid
    print(flim)
    if (flim != 0):
        axi=0
        c = ax[axi].imshow(cutfield(xyf[...].real), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
        axi=1
        c = ax[axi].imshow(cutfield(xyf[...].imag), origin='lower', cmap=cm.bwr, extent=xyextent, interpolation='none', vmin = -flim, vmax=flim)#, vmin=vmin, vmax=vmax, interpolation='none')
        cb = cbax[axi].colorbar(c)
    
    pdf.savefig(f)
    plt.close(f)

4.884527577173636
4.014299462949933
0.0
4.014299462949933
76.57856549650211
0.0
17.66535877620432
0.0
76.5785654965018
3371.402225214826
0.0
295.83270339528923
0.0
295.83270339529287
0.0
3371.402225214809
316241.95080092835
0.0
35191.88550506751
0.0
2589.16826242701
0.0
35191.88550506758
0.0
316241.95080093236
57727499.85722107
0.0
3935506.165849236
0.0
124400.97520823377
0.0
124400.9752082342
0.0
3935506.1658492414
0.0
57727499.857220545
31195630727.315468
0.0
608531592.0212419
0.0
29304422.59088546
0.0
1127036.9451292823
0.0
29304422.590885572
0.0
608531592.0211911
0.0
31195630727.315468
5233005371740.819
0.0
189627678209.96057
0.0
6407389199.400282
0.0
97732781.80135493
0.0
97732781.80135533
0.0
6407389199.400261
0.0
189627678209.9538
0.0
5233005371741.37
